### Import

In [36]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms


### Create Fully Connected Network

In [37]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [38]:
model = NN(784, 10)

In [39]:
x = torch.randn(64, 784)
x

tensor([[ 0.2753,  1.2082, -0.3754,  ...,  0.0707, -1.4083,  1.8515],
        [ 1.6089,  0.1996, -0.2346,  ...,  0.1044, -1.3904, -1.1592],
        [-1.3448, -0.8358, -1.7731,  ..., -1.1251,  0.3740,  0.3189],
        ...,
        [ 0.2701,  0.5712,  1.1159,  ..., -0.4768,  1.3569, -1.4349],
        [ 1.8294,  0.0496, -0.3929,  ..., -2.7146, -0.4047, -0.2489],
        [-0.9305,  0.4922,  0.6949,  ..., -0.1434,  0.4150, -1.3881]])

In [40]:
model(x).shape

torch.Size([64, 10])

### Set Device

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Hyperparameters

In [42]:
input_size = 784
num_classes = 10 
learning_rate = 0.001
batch_size = 64 
num_epochs = 1

### Load Data

In [43]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Initialize Network

In [44]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

### Loss and Optimizer

In [45]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train

In [46]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Get to correct shape
        data = data.reshape(data.shape[0],-1)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient decent or adam step
        optimizer.step() # update weight
        

In [47]:
data.shape

torch.Size([32, 784])

### Evaluation

In [48]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")  
          
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
        
    model.train()

In [49]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 56001 / 60000 with accuracy 93.34
Checking accuracy on test data
Got 9328 / 10000 with accuracy 93.28
